In [1]:
#IMPORTING LIBRARIES

import Feature_Extraction as fe
import Utils

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn import tree

import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer

Using TensorFlow backend.


In [2]:
# load the dataset

data = open('data/tousend_cons_rels.CSV', encoding="ANSI").read()
label, r1, r2, rmix = [], [], [], []
size = 500
for i, line in enumerate(data.split("\n")):
    if i >= size:
        break
    if line != '' and line.isspace() != True:
        content = line.split(';')
        label.append(content[0])
        r1.append(content[1])
        r2.append(content[2])
        rmix.append(content[1]+content[2])

In [3]:
index = 0
size = len(label)
x = []
while index < size:
    x.append(fe.createFBag(r1[index], r2[index]))
    index = index + 1

In [4]:
vec = DictVectorizer()

xx = vec.fit_transform(x).toarray()

In [5]:
yy = []
for l in label:
    if l == 'true':
        yy.append(1)
    else:
        yy.append(-1)

In [6]:
#naive_bayes.MultinomialNB()
#BernoulliNB
#xgboost.XGBClassifier()
#svm.SVC(kernel='linear', C=1, random_state=0)
#tree.DecisionTreeClassifier()
#SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
#GaussianNB()
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [7]:
clf_ME = ensemble.ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf_NC = NearestCentroid()
clf_SVM1 = svm.SVC(kernel='linear', C=1, random_state=0)
clf_SVM2 = svm.SVC(gamma='scale')
clf_LR = linear_model.Ridge(alpha=.5)
clf_SGDC = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

In [15]:
clf = SVC(C=100, gamma=0.001, kernel= 'rbf')
kfold = 10
skf = StratifiedShuffleSplit(n_splits=kfold)
sets = []
for train, test in skf.split(xx, yy):
    sets.append({'x':train, 'y':test})
k = 0
scores = {'precision': [], 'recall': []}
fps_reqs = {'reqs': [], 'freq': []}
fns_reqs = {'reqs': [], 'freq': []}
while k < kfold:
    x_ids = sets[k]['x']
    y_ids = sets[k]['y']
    x_train = xx[x_ids]
    x_test = xx[y_ids]
    y_train, y_test = [], []
    for i in x_ids:
        y_train.append(yy[i])
    for i in y_ids:
        y_test.append(yy[i])
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)
    #pd2 = []
    #for p in predictions:
    #    if p > 0.3:
    #        pd2.append(1)
    #    else:
    #        pd2.append(-1)
    #predictions = pd2
    p = metrics.precision_score(y_test, predictions, pos_label=1)
    r = metrics.recall_score(y_test, predictions, pos_label=1)
    scores['precision'].append(p)
    scores['recall'].append(r)
    k = k + 1
    #ps, ns = getChallengingReqs(y_test, predictions)
    #for req_couple in ps:
    #    fps_reqs = updateDic(fps_reqs, 'reqs', 'freq', req_couple)
    #for req_couple in ns:
    #    fns_reqs = updateDic(fns_reqs, 'reqs', 'freq', req_couple)

In [16]:
p_mean = np.asarray(scores['precision']).mean()
r_mean = np.asarray(scores['recall']).mean()
f1_mean = 2 * p_mean * r_mean / (p_mean + r_mean)
print("Precision: %0.2f (+/- %0.2f)" % (p_mean, np.asarray(scores['precision']).std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (r_mean, np.asarray(scores['recall']).std() * 2))
print("F1: %0.2f" % (f1_mean))

Precision: 0.75 (+/- 0.43)
Recall: 0.56 (+/- 0.47)
F1: 0.64
